# HW2

In [2]:
from gensim import downloader

vec_num = 100
GLOVE_PATH = f'glove-twitter-{vec_num}'
glove_twitter = downloader.load(GLOVE_PATH)


## Model 1: GloVe

In [66]:
import numpy as np

def open_and_split_file(file_path):
    with open(file_path) as f:
        lines = f.readlines()
        words = []
        tags = []
        for line in lines:
            try:
                word, tag = line.rstrip().split("\t")
                word = word.lower()
                if (word not in glove_twitter):
                    words.append(np.zeros(vec_num))
                else:
                    words.append(glove_twitter[word])
                tags.append(0 if tag == "O" else 1)

            except:
                continue
    return words, tags

In [67]:
train_words, train_labels = open_and_split_file("/home/student/hw2/NER_task_in_NLP/data/train.tagged")

In [68]:
dev_words, dev_labels = open_and_split_file("/home/student/hw2/NER_task_in_NLP/data/dev.tagged")

In [69]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(train_words, train_labels)
y_pred = knn.predict(dev_words)

from sklearn.metrics import f1_score
f1_score(dev_labels, y_pred)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.5422804146208401

## Model 2: Feed Forward

In [70]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.optim import Adam
from sklearn.metrics import f1_score

In [71]:
class CustomDataset(Dataset):
    def __init__(self, words, tags):
        # Create a path-to-label dictionary
        self.words, self.tags = words, tags

    def __len__(self):
        return len(self.words)

    def __getitem__(self, index):
        word = self.words[index]
        tag = self.tags[index]
        word = torch.FloatTensor(word).squeeze()
        data = {"word": word, "labels": tag}
        return data

In [72]:
train_dataset = open_and_split_file("/home/student/hw2/NER_task_in_NLP/data/train.tagged")
train_dataset = CustomDataset(train_dataset[0], train_dataset[1])
dev_dataset = open_and_split_file("/home/student/hw2/NER_task_in_NLP/data/dev.tagged")
dev_dataset = CustomDataset(dev_dataset[0], dev_dataset[1])
datasets = {"train": train_dataset, "dev": dev_dataset}

In [73]:
class FeedForwardNN(nn.Module):
    def __init__(self, vec_dim, num_classes, hidden_dim=100):
        super(FeedForwardNN, self).__init__()
        self.first_layer = nn.Linear(vec_dim, hidden_dim)
        self.second_layer = nn.Linear(hidden_dim, num_classes)
        self.activation = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.loss = nn.CrossEntropyLoss()

    def forward(self, word, labels=None):
        x = self.first_layer(word)
        x = self.activation(x)
        x = self.second_layer(x)
        x = self.sigmoid(x)
        if labels is None:
            return x, None
        loss = self.loss(x, labels)
        return x, loss

In [74]:
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader

def train(model, data_sets, optimizer, num_epochs: int, batch_size=16):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_loaders = {"train": DataLoader(data_sets["train"], batch_size=batch_size, shuffle=True),
                    "dev": DataLoader(data_sets["dev"], batch_size=batch_size, shuffle=False)}
    model.to(device)

    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()

        for batch in data_loaders['train']:
            batch_size = 0
            for k, v in batch.items():
                batch[k] = v.to(device)
                batch_size = v.shape[0]

            optimizer.zero_grad()
            _, loss = model(**batch)
            loss.backward()  # The important part
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)  # TODO: Delete when running models 2 and 3.
            optimizer.step()
                
    # Now use the dev dataset to evaluate the model.
    model.eval()
    predictions = torch.tensor([])
    tags = torch.tensor([])
    for batch in data_loaders['dev']:
        batch_size = 0
        for k, v in batch.items():
            batch[k] = v.to(device)
            batch_size = v.shape[0]

        optimizer.zero_grad()    
        with torch.no_grad():
            outputs, _ = model(**batch)
            pred = outputs.argmax(dim=-1).clone().detach().cpu()
            predictions = torch.cat((predictions, pred), 0)
        tags = torch.cat((tags, (batch["labels"].clone().detach().cpu())), 0)
    
    # Filter out padding (where labels are -1)
    flat_predictions = predictions.view(-1)
    flat_tags = tags.view(-1)
    mask = flat_tags != -1
    score = f1_score(flat_tags[mask], flat_predictions[mask])
    print(f'F1 score: {score}')
                
    # with open('model.pkl', 'rb') as f:
    #     model = torch.load(f)
    return model

In [77]:
# Hyper-parameters that we can change:
# hidden_dim: the dimension of the hidden layer
# num_epochs: the number of epochs to train the model
# learning_rate: the learning rate of the optimizer (Adam) - find out more about it in the documentation.
model = FeedForwardNN(vec_num, 2, hidden_dim=int(vec_num*2))
optimizer = Adam(params=model.parameters())
model = train(model=model, data_sets=datasets, optimizer=optimizer, num_epochs=15)

F1 score: 0.5868073878627968


## Model 3: LSTM

In [13]:
from torch.autograd import Variable

''' This code was inspired by the following source: https://cnvrg.io/pytorch-lstm/
    Because we thought it was more suitable for our uses than the one studied in class. '''
   
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, hidden2_size, num_stacked_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes  # We have 2 classes, binary.
        self.input_size = input_size  # The number of expected features in the input x.
        self.hidden_size = hidden_size  # number of features in hidden state.
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)  # lstm
        self.layer1 =  nn.Linear(hidden_size, hidden2_size)  # Layer 1 in the LSTM
        self.layer2 = nn.Linear(hidden2_size, num_classes)  # Layer 2 in the LSTM

        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.loss = nn.CrossEntropyLoss()

    
    def forward(self, word, labels=None):
        word = word.unsqueeze(1)
        h_0 = Variable(torch.zeros(self.num_stacked_layers, word.size(0), self.hidden_size)).to(self.device)  # Short term memory.
        c_0 = Variable(torch.zeros(self.num_stacked_layers, word.size(0), self.hidden_size)).to(self.device)  # Long term memory.
        
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(word, (h_0, c_0))  # Perform lstm with relation to input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size)  # Reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.layer1(out)  # First Dense
        out = self.relu(out)  # Activation function - Relu
        out = self.layer2(out)  # Second layer
        out = self.softmax(out) # Activation function - Softmax
        # pred = outputs.argmax(dim=-1).clone().detach().cpu()
        if labels is None:
            return out, None
        loss = self.loss(out, labels)
        return out, loss

In [14]:
num_epochs = 15
learning_rate = 0.001

num_stacked_layers = 1  # Number of stacked lstm layers, in this model, we do not stack layers.
num_classes = 2  # Number of output classes

lstm = LSTM(num_classes, vec_num, int(vec_num/2), vec_num*2, num_stacked_layers)  # Initiate the model

In [15]:
cross_entropy = nn.CrossEntropyLoss()  # Cross entropy loss
optimizer = Adam(lstm.parameters(), lr=learning_rate)  # Adam optimizer

In [16]:
lstm = train(model=lstm, data_sets=datasets, optimizer=optimizer, num_epochs=15)
print()

/tmp/ipykernel_36533/1454030792.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.softmax(out) # Activation function - Softmax


F1 score: 0.5806451612903225



## Model 4: Competition

In [123]:
import numpy as np

def open_split_file_and_calc_weights(file_path):
    with open(file_path) as f:
        lines = f.readlines()
        sentences = []
        sentences_tags = []
        words_str = []
        words = []
        tags = []
        count_0, count_1 = 0, 0
        weights = [np.zeros(vec_num)]
        for line in lines:
            word = line.rstrip()
            if word == '':
                sentences.append(words_str)
                sentences_tags.append(tags)
                count_0 += tags.count(0)
                count_1 += tags.count(1)
                print("Number of 0s:", count_0)
                print("Number of 1s:", count_1)
                                
                words_str = []
                tags = []
                continue
            vector_added = False
            try:
                word, tag = word.split("\t")
            except:
                continue
            word = word.lower()
            words_str.append(word)
            tags.append(0 if tag == "O" else 1)

        sentences_vectors = []
        for sentence in sentences:
            for w_idx, word in enumerate(sentence):
                if word not in word2idx.keys():
                    vector_added = True
                    word2idx[word] = len(word2idx.keys())

                if (word not in glove_twitter):
                    stemmed_words = list()
                    # try stemming
                    for i in range(min(len(word), 5)):
                        if word[i:] in glove_twitter:
                            stemmed_words.append(glove_twitter[word[i:]])
                        for j in range(1, min(len(word), 5)):
                            if word[i:-j] in glove_twitter:
                                stemmed_words.append(glove_twitter[word[i:-j]])
                    average_on_stemmed = np.mean(np.array(stemmed_words), axis=0) if stemmed_words else np.zeros(vec_num)

                    window_words = list()
                    for i in range(1, 3):
                        if len(words) > i:
                            window_words.append(words[-i])
                        if w_idx < len(words_str) - i:
                            if words_str[w_idx + i] in glove_twitter:
                                window_words.append(glove_twitter[words_str[w_idx + i]])
                    average_on_window = np.mean(np.array(window_words), axis=0) if window_words else np.zeros(vec_num)
            
                    words.append(np.mean(np.array([average_on_stemmed, average_on_window]), axis=0))
                else:
                    words.append(glove_twitter[word])

            max_len = max([len(sentence) for sentence in sentences])
            for i in range(len(words), max_len):
                words.append(np.zeros(vec_num))
            sentences_vectors.append(words)
            words = []

    return np.array(sentences_vectors), sentences_tags, [count_0, count_1]

In [124]:
def open_and_split_test_file(file_path, word2idx):
    with open(file_path) as f:
        lines = f.readlines()
        words = []
        sentences = []
        for line in lines:
            word = line.rstrip()
            if word == '':
                sentences.append(words)
                words = []
                continue
            
            word = word.lower()
            if word in word2idx.keys():
                words.append(word2idx[word])
            else:
                temp_word = None
                for j in range(1, min(len(word), 5)):
                    if temp_word:
                        break
                    if word[:-j] in word2idx.keys():
                        temp_word = word[:-j]
                        break
                    for i in range(min(len(word), 5)):
                        if word[i:-j] in word2idx.keys():
                            temp_word = word[i:-j]
                            break
                words.append(word2idx[temp_word] if temp_word else word2idx["null"])
    return sentences

In [125]:
train_words, train_labels, weights_matrix, tags_frequencies = open_split_file_and_calc_weights("/home/student/hw2/NER_task_in_NLP/data/train.tagged")
dev_words, dev_labels, _, _ = open_split_file_and_calc_weights("/home/student/hw2/NER_task_in_NLP/data/dev.tagged") # open_and_split_dev_test_file("/home/student/hw2/NER_task_in_NLP/data/dev.tagged", train_word2idx)
# "/home/student/hw2/NER_task_in_NLP/data/dev.tagged", train_word2idx)

padded_train_tags = nn.utils.rnn.pad_sequence([torch.tensor(x) for x in train_labels], batch_first=True, padding_value=-1)
padded_dev_tags = nn.utils.rnn.pad_sequence([torch.tensor(x) for x in dev_labels], batch_first=True, padding_value=-1)

Number of 0s: 23
Number of 1s: 4
Number of 0s: 37
Number of 1s: 5
Number of 0s: 48
Number of 1s: 6
Number of 0s: 57
Number of 1s: 6
Number of 0s: 68
Number of 1s: 7
Number of 0s: 87
Number of 1s: 8
Number of 0s: 98
Number of 1s: 8
Number of 0s: 123
Number of 1s: 8
Number of 0s: 127
Number of 1s: 8
Number of 0s: 139
Number of 1s: 8
Number of 0s: 161
Number of 1s: 9
Number of 0s: 173
Number of 1s: 9
Number of 0s: 181
Number of 1s: 9
Number of 0s: 206
Number of 1s: 9
Number of 0s: 219
Number of 1s: 9
Number of 0s: 241
Number of 1s: 10
Number of 0s: 251
Number of 1s: 10
Number of 0s: 275
Number of 1s: 10
Number of 0s: 287
Number of 1s: 10
Number of 0s: 292
Number of 1s: 10
Number of 0s: 310
Number of 1s: 10
Number of 0s: 331
Number of 1s: 11
Number of 0s: 345
Number of 1s: 11
Number of 0s: 356
Number of 1s: 11
Number of 0s: 372
Number of 1s: 11
Number of 0s: 390
Number of 1s: 11
Number of 0s: 406
Number of 1s: 11
Number of 0s: 425
Number of 1s: 11
Number of 0s: 443
Number of 1s: 11
Number 

NameError: name 'word2idx' is not defined

In [128]:
# first we want to create a lstm which will generate new vector representations for the words
class LSTM_Vectorizer(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, hidden2_size, num_stacked_layers, dropout_prob=0.1, bound_on_freeze=12000, batch_size=16):
        super(LSTM_Vectorizer, self).__init__()
        self.num_classes = num_classes  # We have 2 classes, binary.
        self.input_size = input_size  # The number of expected features in the input x.
        self.hidden_size = hidden_size  # number of features in hidden state.
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True, num_layers=self.num_stacked_layers, bidirectional=True, dropout=dropout_prob)  # lstm
        self.layer1 =  nn.Linear(2 * hidden_size, hidden2_size)  # Layer 1 in the LSTM
        self.layer2 = nn.Linear(hidden2_size, num_classes)  # Layer 2 in the LSTM

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.loss = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 10.0]).to(self.device), ignore_index=-1)
        self.counter = 0
        self.bound_on_freeze = bound_on_freeze

        # self.hn = Variable(torch.zeros(self.num_stacked_layers * 2, batch_size, self.hidden_size)).to(self.device)  # Short term memory.
        # self.cn = Variable(torch.zeros(self.num_stacked_layers * 2, batch_size, self.hidden_size)).to(self.device)  # Long term memory.
        
    
    def forward(self, sentence, labels=None):
        sentence = sentence.type(torch.FloatTensor).to(self.device)
        num_layers = self.num_stacked_layers * 2
        h_0 = Variable(torch.zeros(num_layers, sentence.size(0), self.hidden_size)).to(self.device)  # Short term memory.
        c_0 = Variable(torch.zeros(num_layers, sentence.size(0), self.hidden_size)).to(self.device)  # Long term memory.
        
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(sentence, (h_0, c_0))  # Perform lstm with relation to input, hidden, and internal state
        # hn = hn.view(self.num_stacked_layers, 16, self.hidden_size)[-1]
        # self.hn = hn
        # self.cn = cn

        # Reshape output for linear layer
        # output shape: [batch_size=16, max_len=41, 2 * hidden_size]
        batch_size, seq_length, features = output.shape
        output = output.contiguous().view(batch_size * seq_length, features)

        # hn = hn.view(-1, self.hidden_size)  # Reshaping the data for Dense layer next
        out = self.relu(output)
        out = self.layer1(out)  # First Dense
        out = self.relu(out)  # Activation function - Relu
        out = self.layer2(out)  # Second layer
        out = self.sigmoid(out) # Activation function - Softmax
        # pred = outputs.argmax(dim=-1).clone().detach().cpu()
        out = out.view(batch_size, seq_length, -1)
        if labels is None:
            return out, None
        labels = labels.view(-1)
        loss = self.loss(out.view(-1, out.shape[-1]), labels)  # Reshape output to [batch_size * seq_length, num_classes]
        # loss.backward(retain_graph=True)  # Set retain_graph=True to keep the computational graph
        # TODO: The retain graph wasn't there.
        # TODO: Put it back in the train function and remove from here if the clipping doesn't work.
        # nn.utils.clip_grad_norm_(self.parameters(), max_norm=1.0)
        return out, loss

In [129]:
class CompetativeDataset(Dataset):
    def __init__(self, sentences, labels):
        # Create a path-to-label dictionary
        self.sentences = sentences
        self.tags = labels

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        sentence = torch.from_numpy(self.sentences[index]).squeeze()
        tag = self.tags[index]
        data = {"sentence": sentence, "labels": tag}
        return data

In [130]:
train_dataset = CompetativeDataset(sentences=train_words, labels=padded_train_tags)
dev_dataset = CompetativeDataset(sentences=dev_words, labels=padded_dev_tags)
datasets = {"train": train_dataset, "dev": dev_dataset}


In [131]:
num_epochs = 15
learning_rate = 0.001

num_stacked_layers = 5  # Number of stacked lstm layers, in this model, we do not stack layers.
num_classes = 2  # Number of output classes

lstm_model = LSTM_Vectorizer(num_classes, vec_num, int(vec_num/2), vec_num*2, num_stacked_layers)  # Initiate the model

In [132]:
optimizer = Adam(lstm_model.parameters(), lr=learning_rate)  # Adam optimizer

In [133]:
lstm_model = train(model=lstm_model, data_sets=datasets, optimizer=optimizer, num_epochs=num_epochs)
print()

F1 score: 0.6254098360655738

